In [1]:
import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch.utils.data import DataLoader
from torch_geometric.utils import to_dense_adj

In [2]:
from helpers import CVFConfigDataset

In [3]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        x = torch.sparse.mm(adjacency, x)
        return x

In [4]:
dataset = CVFConfigDataset()
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

In [5]:
# edge_index = torch.tensor([
#     [0, 1],
#     [0, 2],
#     [0, 3],
#     [1, 0],
#     [2, 0],
#     [3, 0]
# ], dtype=torch.long)
edge_index = torch.tensor(
    [
        [0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [0, 5],
        [0, 6],
        [0, 7],
        [0, 8],
        [0, 9],
        [1, 0],
        [2, 0],
        [3, 0],
        [4, 0],
        [5, 0],
        [6, 0],
        [7, 0],
        [8, 0],
        [9, 0],
    ],
    dtype=torch.long,
)


# # x = torch.tensor([[0], [0], [0], [0]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]
# x = torch.tensor([[0, 1], [0, 1], [0, 1], [0, 1]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]

# y = torch.tensor([[3.0, 1.0]]) # ranks

# data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y)
# data.train_mask = torch.tensor([1 for _ in range(len(x))])

In [6]:
adjacency = to_dense_adj(edge_index.t().contiguous())[0]
adjacency += torch.eye(len(adjacency))
# adjacency = adjacency.repeat(data_loader.batch_size, 1)
adjacency

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [ ]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gnn1 = VanillaGNNLayer(dim_in, dim_h)
        self.gnn2 = VanillaGNNLayer(dim_h, dim_out)
        # self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x, adjacency):
        h = self.gnn1(x, adjacency)
        h = torch.relu(h)
        h = self.gnn2(h, adjacency)
        h = torch.relu(h)
        # print("h2 r", h)
        # h = self.out(h)
        # print(torch.relu(h))
        # result = F.softmax(h, dim=0)
        # print("result", result)
        h = h.flatten().unsqueeze(1)
        return h

    def fit(self, data_loader, epochs):
        # criterion = torch.nn.MSELoss()
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001, weight_decay=5e-4)
        self.train()
        for epoch in range(epochs + 1):
            avg_loss = 0
            count = 0
            for batch in data_loader:
                # x = batch[0].squeeze()
                # print(x)
                x = batch[0][0]
                # y = torch.zeros(4, 1)
                # y[batch[1]] = 1.0
                y = batch[1]
                optimizer.zero_grad()
                out = self(x, adjacency).T
                # print("output", out, "y", y)
                loss = criterion(out, y)
                avg_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            loss = avg_loss / count
            print("Loss:", loss)

            # if loss < 0.009:
            #     print("Loss threshold met.")
            #     break

In [ ]:
gnn = VanillaGNN(data_loader.batch_size, 64, data_loader.batch_size)
print(gnn)
gnn.fit(data_loader, epochs=50)

VanillaGNN(
  (gnn1): VanillaGNNLayer(
    (linear): Linear(in_features=1, out_features=64, bias=False)
  )
  (gnn2): VanillaGNNLayer(
    (linear): Linear(in_features=64, out_features=1, bias=False)
  )
)
Loss: tensor(2.3074, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=<DivBackward0>)
Loss: tensor(2.3027, grad_fn=

In [ ]:
# out = gnn(torch.tensor([[3.], [2.], [2.], [2.]]), adjacency)
# print(out)

In [ ]:
for batch in data_loader:
    x = batch[0][0]
    y = batch[1]
    predicted = gnn(x, adjacency).argmax(dim=0)
    print("y", y, "predicted", predicted)

y tensor([9]) predicted tensor([0])
y tensor([8]) predicted tensor([0])
y tensor([8]) predicted tensor([0])
y tensor([7]) predicted tensor([0])
y tensor([8]) predicted tensor([0])
y tensor([7]) predicted tensor([0])
y tensor([7]) predicted tensor([0])
y tensor([6]) predicted tensor([0])
y tensor([8]) predicted tensor([0])
y tensor([7]) predicted tensor([0])
y tensor([7]) predicted tensor([0])
y tensor([6]) predicted tensor([0])
y tensor([7]) predicted tensor([0])
y tensor([6]) predicted tensor([0])
y tensor([6]) predicted tensor([0])
y tensor([5]) predicted tensor([0])
y tensor([8]) predicted tensor([0])
y tensor([7]) predicted tensor([0])
y tensor([7]) predicted tensor([0])
y tensor([6]) predicted tensor([0])
y tensor([7]) predicted tensor([0])
y tensor([6]) predicted tensor([0])
y tensor([6]) predicted tensor([0])
y tensor([5]) predicted tensor([0])
y tensor([7]) predicted tensor([0])
y tensor([6]) predicted tensor([0])
y tensor([6]) predicted tensor([0])
y tensor([5]) predicted tens